Yelp Covid Banner to classify Temporary Closed

In [41]:
import pandas as pd
import numpy as np
import os

In [42]:
os.chdir('/Documents/MSIS/Fall 2020/Team Project')
os.getcwd()

'/Documents/MSIS/Fall 2020/Team Project'

In [43]:
dataset = pd.read_csv('yelpState.csv')
dataset.head()

,business_id,CovidBanner,TemporaryClosedUntil,IsOpen,State
0,--1UhMGODdWsrMastO9DZw,FALSE,FALSE,1,AB
1,--6MefnULPED_I942VcFNA,FALSE,FALSE,1,ON
2,--7zmmkVg-IMGaXbuVd0SQ,FALSE,FALSE,1,NC
3,--8LPVSo5i0Oo61X01sV9A,FALSE,FALSE,1,AZ
4,--9QQLMTbFzLJ_oT-ON3Xw,FALSE,FALSE,1,AZ


In [44]:
train = dataset[dataset['CovidBanner'] != 'FALSE'][['CovidBanner','TemporaryClosedUntil']].dropna()
train.head()

,CovidBanner,TemporaryClosedUntil
22,We are back! This Sport Clips location is offi...,FALSE
52,We are seeing patients both virtually and in p...,FALSE
83,Hola amigos! The Cantina Location is Now open!...,FALSE
109,"Due to some recent positive COVID-19 cases, th...",2020-07-06T00:00:00.000Z
118,All 7 Stores Are Open!\nMonday - Friday 9:00am...,FALSE


In [45]:
train_X = train['CovidBanner']
train_X.head()

22     We are back! This Sport Clips location is offi...
52     We are seeing patients both virtually and in p...
83     Hola amigos! The Cantina Location is Now open!...
109    Due to some recent positive COVID-19 cases, th...
118    All 7 Stores Are Open!\nMonday - Friday 9:00am...
Name: CovidBanner, dtype: object

In [47]:
train.loc[train['TemporaryClosedUntil'] != 'FALSE', 'TemporaryClosedUntil'] = 'TRUE'
train_Y = train['TemporaryClosedUntil']
train_Y.head()

22     FALSE
52     FALSE
83     FALSE
109     TRUE
118    FALSE
Name: TemporaryClosedUntil, dtype: object

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [49]:

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])
text_clf = text_clf.fit(train_X, train_Y)

In [50]:
predicted = text_clf.predict(train_X)
np.mean(predicted == train_Y)

0.9711873194578179

In [51]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)), ])
text_clf_svm = text_clf_svm.fit(train_X, train_Y)
predicted_svm = text_clf_svm.predict(train_X)
np.mean(predicted_svm == train_Y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.9717798681579143

In [52]:
train['predict_svm'] = predicted_svm

In [53]:
train['predict_NNB'] = predicted

In [54]:
train[train['TemporaryClosedUntil'] == 'TRUE'][train['predict_NNB'] == 'TRUE']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,CovidBanner,TemporaryClosedUntil,predict_svm,predict_NNB
65107,Attention - Nos opérations sont actuellement f...,TRUE,FALSE,TRUE
76902,Nos centres sont présentement fermés suite au ...,TRUE,FALSE,TRUE
185742,"Valued Clients,\n\nFollowing the responsible d...",TRUE,FALSE,TRUE


In [55]:
from sklearn import metrics

In [57]:
f1 = metrics.f1_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
p = metrics.precision_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
r = metrics.recall_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
print('Performance for Naive Baysien:')
print("f1:"+str(f1))
print("precision:"+str(p))
print("recall:"+str(r))

Performance for Naive Baysien:
f1:0.5002848303458736
precision:0.5035178005569426
recall:0.5931293626666949


In [58]:
f1 = metrics.f1_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
p = metrics.precision_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
r = metrics.recall_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
print('Performance for SVM:')
print("f1:"+str(f1))
print("precision:"+str(p))
print("recall:"+str(r))

Performance for SVM:
f1:0.4928439953420233
precision:0.5
recall:0.48588993407895714


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
